<a href="https://colab.research.google.com/github/naokityokoyama/HDC/blob/main/HDC_vs_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install unidecode num2words evaluate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=2ac120039fb1589271974d63928a9a4499fcb3e324d6460db15f2e314c4856e9
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt


In [3]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertForSequenceClassification
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

import re
import zipfile
from unidecode import unidecode
import string
from num2words import num2words

from tqdm import tqdm
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import evaluate

In [4]:
path_origin = '/content/drive/MyDrive/uff/fake.zip'
path_destino = '/content/'
with zipfile.ZipFile(path_origin, "r") as zip_ref:
    zip_ref.extractall(path_destino)

In [5]:
#build dataset

df_fake = pd.read_csv('/content/fakes.csv')[['text']]
df_fake['target'] = 1
df_true = pd.read_csv('/content/true.csv')[['text']]
df_true['target'] = 0
df = pd.concat([df_fake, df_true]).reset_index(drop=True)

In [6]:
# Definir o tamanho da amostra
sample_size = 10000

# Criar uma amostra balanceada
df = df.groupby("target", group_keys=False).apply(lambda x: resample(x, n_samples=sample_size // df["target"].nunique(), random_state=42))
df = df.reset_index(drop=True)

/tmp/ipython-input-119938293.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("target", group_keys=False).apply(lambda x: resample(x, n_samples=sample_size // df["target"].nunique(), random_state=42))


In [7]:
#clean

def n2w(texto:str)->str:
  padrao = r"\d+"
  numeros = re.findall(padrao, texto)
  for numero in numeros:
    extenso = num2words(numero, lang='pt')
    texto = texto.replace(numero, extenso)
  return texto

In [8]:
for repet in tqdm(range(2)):  #bug para rodar 2x
  df['text'] = df['text'].str.lower()
  df['text'] = df['text'].str.replace(f"[{string.punctuation}]", "", regex=True)
  df['text'] = df['text'].apply(lambda x: ' '.join(x.split()))
  df['text'] = df['text'].str.replace('"', '').str.replace('\\', '')
  df['text'] = df['text'].apply(n2w)
  df['text'] = df['text'].apply(unidecode)

100%|██████████| 2/2 [00:01<00:00,  1.77it/s]


In [9]:
#create X and y
X = df['text']
y = df['target']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [11]:
train = Dataset.from_dict({'text': X_train, 'labels': y_train})
val = Dataset.from_dict({'text': X_val, 'labels': y_val})
test = Dataset.from_dict({'text': X_test, 'labels': y_test})
dataset_dict = DatasetDict({'train': train, 'test': test})
dataset_dict = DatasetDict({'train': train, 'validation': val, 'test': test})
num_labels = len(set(np.array(y)))

In [12]:
# define pre-trained model path
model_path = "google-bert/bert-base-uncased"

# load model tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=num_labels)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# freeze all base model parameters
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [15]:
# define text preprocessing
def preprocess_function(examples):
    # return tokenized text with truncation
    return tokenizer(examples["text"], truncation=True)

# preprocess all datasets
tokenized_data = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [16]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [17]:
# load metrics
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred

    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1,
                                                                 keepdims=True)
    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]
    # compute auc
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs,
                                     references=labels)['roc_auc'],3)

    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes,
                                     references=labels)['accuracy'],3)

    return {"Accuracy": acc, "AUC": auc}

In [18]:
# hyperparameters
lr = 2e-4
batch_size = 32
num_epochs = 2

training_args = TrainingArguments(
    output_dir="bert-phishing-classifier_teacher",
    report_to="none",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipython-input-2732273287.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.671900,0.643112,0.631000,0.680000
2,0.649800,0.637162,0.647000,0.695000


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


TrainOutput(global_step=376, training_loss=0.6608807584072681, metrics={'train_runtime': 118.9644, 'train_samples_per_second': 100.87, 'train_steps_per_second': 3.161, 'total_flos': 761838060794880.0, 'train_loss': 0.6608807584072681, 'epoch': 2.0})

In [20]:
# apply model to validation dataset
predictions = trainer.predict(tokenized_data["validation"])

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
metrics = compute_metrics((logits, labels))
print(metrics)



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'Accuracy': np.float64(0.638), 'AUC': np.float64(0.699)}
